In [1]:
import time
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline

In [2]:
inicio = '1981-06-30'
fin = '2016-08-30'

In [3]:
# Descargar datos de comisiones (Lento)
rng = pd.date_range(start=inicio, end=fin, freq='MS')
dfs = []
for day in rng:
    url = 'http://www.safp.cl/safpstats/stats/apps/estcom/estcom.php?fecha=%s'%day.strftime("%Y%m%d")
    df = pd.read_html(url)[19].loc[3:8,0:1]
    df['date'] = day
    dfs.append(df.pivot(index='date', columns=0, values=1))
    time.sleep(1)

In [4]:
df_s = pd.concat(dfs)
df_s.drop(labels=[u'(*)'],axis=1, inplace=True)
df_s.tail()

,CAPITAL,CUPRUM,HABITAT,MODELO,PLANVITAL,PROVIDA
date,,,,,,
2016-04-01,1.44,1.48,1.27,0.77,0.47,1.54
2016-05-01,1.44,1.48,1.27,0.77,0.47,1.54
2016-06-01,1.44,1.48,1.27,0.77,0.47,1.54
2016-07-01,1.44,1.48,1.27,0.77,0.47,1.54
2016-08-01,1.44,1.48,1.27,0.77,0.41,1.54


In [5]:
# Opciones lectura y escritura de archivos csv compatibles con excel spanish
karg_csvw = dict(sep=';', decimal=',')
df = df_s.apply(pd.to_numeric, errors='coerce').div(100.)
df.to_csv('comisiones/comisiones.csv', **karg_csvw)
df.tail()

,CAPITAL,CUPRUM,HABITAT,MODELO,PLANVITAL,PROVIDA
date,,,,,,
2016-04-01,0.0144,0.0148,0.0127,0.0077,0.0047,0.0154
2016-05-01,0.0144,0.0148,0.0127,0.0077,0.0047,0.0154
2016-06-01,0.0144,0.0148,0.0127,0.0077,0.0047,0.0154
2016-07-01,0.0144,0.0148,0.0127,0.0077,0.0047,0.0154
2016-08-01,0.0144,0.0148,0.0127,0.0077,0.0041,0.0154
